# Bibliotecas

In [6]:
import sympy as sp
import math

# Criação das variáveis simbólicas

## Material e geometria

In [7]:
A, E, L, theta = sp.symbols(['A', 'E', 'L', 'theta'])

## Forças e deslocamentos

In [8]:
f1, f2, f3, f4, f5, f6, f7, f8 = sp.symbols(['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8'])
d1, d2, d3, d4, d5, d6, d7, d8 = sp.symbols(['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8'])

## Seno e cosseno

In [9]:
cosseno = sp.cos(theta)
seno = sp.sin (theta)

# Criação das matrizes e vetores

## Matriz de transformação de coordenada

In [10]:
T = sp.Matrix([[cosseno, seno, 0, 0], [0, 0, cosseno, seno]])

## Matriz de rigidez do elemento em coordenadas globais

In [11]:
Ke_= sp.Matrix([[E*A/L, - E*A/L], [- E*A/L, E*A/L]])

## Matriz de rigidez do elemento em coordenadas globais

In [12]:
Ke = T.T*Ke_*T

## Vetores de forças e deslocamentos

In [13]:
f = sp.Matrix([f1, f2, f3, f4, f5, f6, f7, f8])
d = sp.Matrix([d1, d2, d3, d4, d5, d6, d7, d8])

# Criação de uma função que monta um elemento de treliça

In [14]:
def EL_TRELICA(E_input, A_input, L_input, theta_input, GL1, GL2, GL3, GL4):
    theta_input = theta_input*math.pi/180
    K_elemento = Ke.subs([(E,E_input), (A,A_input) , \
                         (L,L_input) , (theta,theta_input)])
    K_elemento = K_elemento.row_insert(0,sp.Matrix([[GL1, GL2, GL3, GL4]]))
    K_elemento = K_elemento.col_insert(0,sp.Matrix([0,GL1, GL2, GL3, GL4]))
#O elemento sairá com os índices dos GDLs
#col_insert tem menos []
    return(K_elemento)

# Elementos da treliça 

In [15]:
K_ELEM = []

K_ELEM.append(EL_TRELICA(200000,25,500,90.,1,2,3,4))
K_ELEM.append(EL_TRELICA(200000,25,500,0.,3,4,5,6))
K_ELEM.append(EL_TRELICA(200000,25,500,-90.,5,6,7,8))
K_ELEM.append(EL_TRELICA(200000,25,500,180.,7,8,1,2))
K_ELEM.append(EL_TRELICA(200000,25,500*math.sqrt(2),135.,7,8,3,4))

# Montagem da matriz de rigidez global mediante espalhamento

In [16]:
n_de_elementos = len(K_ELEM)
n_de_GDL = 8
K_STRUCT = sp.Matrix.zeros(n_de_GDL,n_de_GDL)
for n in range(n_de_elementos):
    #Extrai os GDLs do elemento e armazena na lista GDL
    GDL = []
    GDL.append(K_ELEM[n][0,1] - 1)
    GDL.append(K_ELEM[n][0,2] - 1)
    GDL.append(K_ELEM[n][0,3] - 1)
    GDL.append(K_ELEM[n][0,4] - 1)

    #Preenche a matriz de rigidez global K_STRUCT
    for i in range(len(GDL)):
        for j in range(len(GDL)):
            K_STRUCT[GDL[i],GDL[j]] += K_ELEM[n][i+1,j+1]

# Resolução do exercício

## Definição dos valores nos vetores de forças e deslocamentos globais

In [28]:
f = f.subs([(f4,0),(f5,0),(f6,-1000),(f7,0),(f8,0)])
f_T=f.T

d = d.subs([(d1,0),(d2,0),(d3,0)])
d_T = d.T
system = K_STRUCT*d-f

In [27]:
system

Matrix([
[                                               -6.12323399573677e-13*d4 - 10000.0*d7 + 1.22464679914735e-12*d8 - f1],
[                                               -10000.0*d4 + 1.22464679914735e-12*d7 - 1.49975978266186e-28*d8 - f2],
[                                   -3535.53390593274*d4 - 10000*d5 - 3535.53390593274*d7 + 3535.53390593274*d8 - f3],
[                                                    13535.5339059327*d4 + 3535.53390593274*d7 - 3535.53390593274*d8],
[                           10000.0*d5 - 6.12323399573677e-13*d6 - 3.74939945665464e-29*d7 + 6.12323399573677e-13*d8],
[                                -6.12323399573677e-13*d5 + 10000.0*d6 + 6.12323399573677e-13*d7 - 10000.0*d8 + 1000],
[3535.53390593274*d4 - 3.74939945665464e-29*d5 + 6.12323399573677e-13*d6 + 13535.5339059327*d7 - 3535.53390593274*d8],
[            -3535.53390593274*d4 + 6.12323399573677e-13*d5 - 10000.0*d6 - 3535.53390593274*d7 + 13535.5339059327*d8]])

In [32]:
#d_sol = K_STRUCT.LUsolve(f)

In [18]:
def extrair_coeficientes(matriz_K, vetor_d, vetor_f, variaveis_d, variaveis_f):  ##############TRABALHAR NESSE####################
    """
    Extrai coeficientes de K*d - f
    """
    sist = matriz_K*vetor_d-vetor_f
    sist = sp.expand(sist)

    system_coefs = []

    for eq in sist:
        linha = []

        # coeficientes das variáveis de d
        for var in variaveis_d:
            linha.append(eq.coeff(var))

        # termo constante
        constante = eq
        for var in variaveis_d + variaveis_f:
            constante = constante.subs(var, 0)
        linha.append(constante)

        # coeficientes das variáveis de f (com sinal)
        for var in variaveis_f:
            linha.append(eq.coeff(var))

        system_coefs.append(linha)

    return system_coefs

In [42]:
#extrair_coeficientes(K_STRUCT, d, f, [d4, d5, d6, d7, d8],[f1,f2,f3])

# Solução do sistema linear de equações pela regra de Cramer

In [43]:
def cramer(system_coefs):
    if len(system_coefs) != 8 or any(len(linha) != 9 for linha in system_coefs):
        raise ValueError("A entrada deve ser uma matriz 8x9.")

    # Matriz dos coeficientes
    A = sp.Matrix([linha[:-1] for linha in system_coefs])

    # Vetor independente (-b)
    b = sp.Matrix([-linha[-1] for linha in system_coefs])

    delta = A.det()

    if delta == 0:
        return "Sistema impossível ou indeterminado (determinante zero)."

    solucoes = {}

    for i in range(8):
        Ai = A.copy()
        Ai[:, i] = b
        solucoes[f"x{i+1}"] = sp.simplify(Ai.det() / delta)

    return solucoes

In [44]:
cramer(extrair_coeficientes(K_STRUCT, d, f, [d4, d5, d6, d7, d8],[f1,f2,f3]))

{'x1': 0.000100000000000000,
 'x2': 6.12323399573676e-21,
 'x3': 0.000582842712474619,
 'x4': 0.000100000000000000,
 'x5': 0.000482842712474619,
 'x6': -0.00100000000000000,
 'x7': -1.00000000000000,
 'x8': -1.00000000000000}

In [61]:
###sp.solve(system,(f1,f2,f3,d4,d5,d6,d7,d8))

{d4: -0.100000000000002,
 d5: -6.12323399573677e-18,
 d6: -0.582842712474629,
 d7: -0.100000000000002,
 d8: -0.482842712474629,
 f1: 1000.00000000002,
 f2: 1000.00000000002,
 f3: -1000.00000000002}